In [ ]:
# Hướng dẫn về bộ dữ liệu này được thực hiện trong Lab Tokenizer
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json

import json

with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

--2024-02-07 15:45:03--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 74.125.137.207, 2607:f8b0:4023:c0d::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2024-02-07 15:45:03 (195 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
datastore[0]

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
 'is_sarcastic': 0}

In [ ]:
dataset = []
label_dataset = []

for item in datastore:
  dataset.append(item['headline'])
  label_dataset.append(item['is_sarcastic'])

In [ ]:
train_size = int(0.8 * len(dataset))

train_sentence = dataset[:train_size]
test_sentence = dataset[train_size:]

train_label = label_dataset[:train_size]
test_label = label_dataset[train_size:]

In [ ]:
train_sentence

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way',
 "advancing the world's women",
 'the fascinating case for eating lab-grown meat',
 'this ceo will send your kids to school, if you work for his company',
 'top snake handler leaves sinking huckabee campaign',
 "friday's morning email: inside trump's presser for the ages",
 'airline passengers tackle man who rushes cockpit in bomb threat',
 'facebook reportedly working on healthcare features and apps',
 "north korea praises trump and urges us voters to reject 'dull hillary'",
 "actually, cnn's jeffrey lord has been 'indefensible' for a while",
 'barcelona holds huge protest in su

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Train

In [ ]:
vocab_size = len(train_sentence)
embedding_size = 64
max_length = 25


In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = '<OOV>')
tokenizer.fit_on_texts(train_sentence)

In [ ]:
tokenizer.index_word

{1: '<OOV>',
 2: 'to',
 3: 'of',
 4: 'the',
 5: 'in',
 6: 'for',
 7: 'a',
 8: 'on',
 9: 'and',
 10: 'with',
 11: 'is',
 12: 'new',
 13: 'trump',
 14: 'man',
 15: 'from',
 16: 'at',
 17: 'about',
 18: 'you',
 19: 'by',
 20: 'this',
 21: 'after',
 22: 'up',
 23: 'out',
 24: 'be',
 25: 'how',
 26: 'that',
 27: 'it',
 28: 'as',
 29: 'not',
 30: 'are',
 31: 'your',
 32: 'what',
 33: 'his',
 34: 'all',
 35: 'he',
 36: 'who',
 37: 'just',
 38: 'has',
 39: 'will',
 40: 'more',
 41: 'into',
 42: 'one',
 43: 'year',
 44: 'report',
 45: 'have',
 46: 'over',
 47: 'area',
 48: 'why',
 49: 'donald',
 50: 'u',
 51: 'day',
 52: 'can',
 53: 'says',
 54: 's',
 55: 'first',
 56: 'woman',
 57: 'time',
 58: 'like',
 59: 'get',
 60: 'her',
 61: 'old',
 62: "trump's",
 63: 'no',
 64: 'now',
 65: 'off',
 66: 'an',
 67: 'life',
 68: 'obama',
 69: 'people',
 70: 'than',
 71: 'women',
 72: "'",
 73: 'house',
 74: 'back',
 75: 'was',
 76: 'still',
 77: 'white',
 78: 'make',
 79: 'down',
 80: 'clinton',
 81: 'my',

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentence) # chuyển sang viết bằng số

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_sentence)

In [ ]:
# padding câu

padded_train_sequences  = pad_sequences(train_sequences, maxlen = max_length,truncating = 'post', padding = 'post')
padded_test_sequences  = pad_sequences(test_sequences, maxlen = max_length,truncating = 'post', padding = 'post')

In [ ]:
padded_train_sequences.shape, padded_test_sequences.shape

((21367, 25), (5342, 25))

Trainning

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, RNN, Dense, LSTM, Bidirectional, SimpleRNN
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import tensorflow as tf

In [ ]:
opt = Adam(learning_rate=0.0001)

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [ ]:
model4 = Sequential()
model4.add(Embedding(vocab_size, embedding_size, input_length = max_length))
# model4.add(Bidirectional(LSTM(128, return_sequences=True)))
# model4.add(Dropout(0.2))
model4.add(LSTM(16))
model4.add(Dropout(0.1))
model4.add(Dense(16, activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(1, activation='sigmoid'))

In [ ]:
model4.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 25, 64)            1367488   
                                                                 
 lstm_24 (LSTM)              (None, 16)                5184      
                                                                 
 dropout_16 (Dropout)        (None, 16)                0         
                                                                 
 dense_24 (Dense)            (None, 16)                272       
                                                                 
 dropout_17 (Dropout)        (None, 16)                0         
                                                                 
 dense_25 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1372961 (5.24 MB)
Trainable params: 137

In [ ]:

model4.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

In [ ]:
import numpy as np
train_label = np.array(train_label)


In [ ]:
model4.fit(padded_train_sequences,train_label, epochs=10, validation_data = (padded_test_sequences, np.array(test_label)))

Epoch 1/10
668/668 [==============================] - 15s 19ms/step - loss: 0.6754 - acc: 0.5589 - val_loss: 0.5231 - val_acc: 0.7304
Epoch 2/10
668/668 [==============================] - 6s 9ms/step - loss: 0.4346 - acc: 0.8440 - val_loss: 0.3810 - val_acc: 0.8489
Epoch 3/10
668/668 [==============================] - 5s 7ms/step - loss: 0.2803 - acc: 0.9042 - val_loss: 0.3539 - val_acc: 0.8532
Epoch 4/10
668/668 [==============================] - 6s 8ms/step - loss: 0.1945 - acc: 0.9411 - val_loss: 0.3705 - val_acc: 0.8521
Epoch 5/10
668/668 [==============================] - 5s 7ms/step - loss: 0.1378 - acc: 0.9617 - val_loss: 0.4090 - val_acc: 0.8482
Epoch 6/10
668/668 [==============================] - 5s 7ms/step - loss: 0.1004 - acc: 0.9756 - val_loss: 0.4843 - val_acc: 0.8452
Epoch 7/10
668/668 [==============================] - 5s 7ms/step - loss: 0.0741 - acc: 0.9844 - val_loss: 0.5695 - val_acc: 0.8431
Epoch 8/10
668/668 [==============================] - 5s 7ms/step - loss: 